# Thicket and Extra-P: Thicket Tutorial

Thicket is a python-based toolkit for Exploratory Data Analysis (EDA) of parallel performance data that enables performance optimization and understanding of applications’ performance on supercomputers. It bridges the performance tool gap between being able to consider only a single instance of a simulation run (e.g., single platform, single measurement tool, or single scale) and finding actionable insights in multi-dimensional, multi-scale, multi-architecture, and multi-tool performance datasets.

#### NOTE: An interactive version of this notebook is available in the Binder environment.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/llnl/thicket-tutorial/develop)

# Thicket Modeling Example

This notebook provides an example for using Thicket's modeling feature. The modeling capability relies on _Extra-P_ - a tool for empirical performance modeling. It can perform N-parameter modeling with up to 3 parameters (N <= 3). The models follow a so-called _Performance Model Normal Form (PMNF)_ that expresses models as a summation of polynomial and logarithmic terms. One of the biggest advantages of this modeling method is that the produced models are human-readable and easily understandable.

***

## 1. Import Necessary Packages

To explore the capabilities of thicket with Extra-P, we begin by importing necessary packages.

In [ ]:
import sys

import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from IPython.display import HTML

import thicket as th
import hatchet as ht
from thicket.model_extrap import ExtrapInterface
from thicket.model_extrap import multi_display

display(HTML("<style>.container { width:80% !important; }</style>"))

## 2. Define Dataset Paths and Names

In this example, we use an MPI scaling study, profiled with Caliper, that has metadata about the runs. The data is also already aggregated, which means we can provide the data to Extra-P as-is.

In [ ]:
data = "../data/lulesh/"
thicket = th.Thicket.from_caliperreader(data)

Specifically, the metadata table for this set of profiles contains a `jobsize` column, which provides the amount of cores used for each profile.

In [ ]:
thicket.metadata

## 3. More Information on a Function
***
You can use the `help()` method within Python to see the information for a given object. You can do this by typing `help(object)`. 
This will allow you to see the arguments for the function, and what will be returned. An example is below.

In [ ]:
help(ExtrapInterface)

## 3. Create Models

First, we instatiate an Extra-P interface to create performance models and more.

In [ ]:
extrap_interface = ExtrapInterface()

Then, we create the performance models by passing the thicket object that contains the performance measurements to the `create_models()` function of the `ExtrapInterface`. In order to create the models the interface requires some more information. First, we need to provide the names of the parameters that should be considered for modeling, e.g., the `jobsize`. The `create_models()` function will grab this column from the metadata table to use as our parameter. We also sub-select some metrics, since this dataset has a lot of metrics (otherwise the modeling will take a long time to do all metrics). Finally, we provide a name for the model configuration that the function is going to create for us. Extra-P will internally use the provided name as a unique identifier for modeling experiments. This will come in handy when experimenting with different modeling parameters, metrics, and modeler configurations.

In [ ]:
extrap_interface.create_models(thicket, 
                               parameters=[
                                   "jobsize",
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)",
                                   ],
                               model_name="config1")

### Creating Multi-Parameter Models

Extra-P support multi-parameter modeling. Consequently you can analyze multiple application parameters with Thicket and Extra-P. To create performance models considering multiple parameters simply provide them for the `parameters` variable of the `create_models()` function. The code below shows an example.

In [ ]:
thicket_multi = th.Thicket.from_caliperreader(data)
extrap_interface_multi = ExtrapInterface()
extrap_interface_multi.create_models(thicket_multi, 
                               parameters=[
                                   "jobsize",
                                   "problem_size"
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)",
                                   ],
                               model_name="config1")

## 4. Models Dataframe

The created performance models including some statistical quality control metrics such as the RSS (residual sum of squares) or the SMAPE (symmetric mean absolute percentage error) are stored in thicket's aggregated statistics table.

In [ ]:
thicket.statsframe.dataframe

## 5. Show the Models Dataframe with Embedded Plots

(For every `node`, sub-selected `metric` combination)

Besides the thicket object containing the models one can provide a variety of plotting options to the `to_html()` function that will change the displayed plots. One can for example select between displaying the mean, median, min, max measured metric values. Furthermore, one can display statistical values such as the RSS and SMAPE.

In [ ]:
with pd.option_context("display.max_colwidth", 1):
    display(HTML(extrap_interface.to_html(thicket, show_mean=True, show_median=True, show_min_max=True, RSS=True, SMAPE=True)))

### Displaying Multi-Parameter Data

To display multi-parameter models one can simply use the same functions as for single-parameter models. The method itself will figure out how many model parameters exist and choose the correct function to display your results.

In [ ]:
with pd.option_context("display.max_colwidth", 1):
    display(HTML(extrap_interface_multi.to_html(thicket_multi, show_mean=True, show_median=True, show_min_max=True, RSS=True, SMAPE=True)))

### Displaying multiple models in one plot

To display multiple models in one plot one case use the `multi_display()` function.

We can for example filter the dataframe for all kernels that belong to MPI operations.
We then take the models of specific kernels, e.g., the first two, and pass them into the `multi_display()` function.
The `multi_display()` function then displays both models in the same plot. This works for models with multiple parameter as shown in the example below but also for models with only a single parameter.

In [ ]:
model_objects = thicket_multi.statsframe.dataframe["Avg time/rank (exc)_extrap-model"].filter(like="MPI_")
    
models = []
max = 2
counter = 0
for model in model_objects:
    if counter < max:
        models.append(model)
    counter += 1

plt.clf()
fig, ax = multi_display(models)
plt.show()
plt.close()


## 6. Query Specific Model

The last node `{"name": "MPI_Allreduce", "type": "function"}`, has an interesting graph so we want to retrieve its model. This can be achieved by indexing the `models_df` DataFrame for our chosen node for the metric `Avg time/rank (exc)_extrap-model`.

In [ ]:
model_obj = thicket.statsframe.dataframe.at[thicket.statsframe.dataframe.index[len(thicket.statsframe.dataframe.index)-1], "Avg time/rank (exc)_extrap-model"]

You can also get the function of a model via the model object as shown below.

In [ ]:
model_obj.mdl.hypothesis.function.to_string()

## 7. Operations on a model

To predict a metric value for a specific configuration of the chosen model parameters, we can evaluate the model like a function by simply providing the values of the chosen parameters.

In [ ]:
model_obj.eval(600)

### Displaying the model:

It returns a _figure_ and an _axis_ objects. The axis object can be used to adjust the plot, i.e., change labels. The `display()` function features several optional input variables that change they way how the data is displayed. For example we can set the `RSS` (bool) value, that determines whether to display Extra-P RSS on the plot. Furthermore, we can show the min, max, mean, and median measured metric values again. Finally, we have the option to display an "optimal" scaling model. If one has an idea how the function should scale like, this expectation function can be enetered as shown in the example code below. The plot then shows the expected metric value compared to the created Extra-P model, ready for comparison.

In [ ]:
plt.clf()
fig, ax = model_obj.display(show_mean=True, show_median=True, 
                            show_min_max=True, 
                            RSS=True, SMAPE=True, show_opt_scaling=True,
                            opt_scaling_func="log2(p)**1")
plt.show()
plt.close()

The same is true when plotting the results of a multi-parameter model. An example is shown below.

In [ ]:
model_obj_multi = thicket_multi.statsframe.dataframe.at[thicket_multi.statsframe.dataframe.index[len(thicket_multi.statsframe.dataframe.index)-1], "Avg time/rank (exc)_extrap-model"]
plt.clf()
fig, ax = model_obj_multi.display(show_mean=True, show_median=True, 
                            show_min_max=True, 
                            RSS=True, SMAPE=True, show_opt_scaling=True,
                            opt_scaling_func="q*log2(p)**1")
plt.show()
plt.close()

If you want an interactive matplotlib chart you can set `%matplotlib widget` in the notebook.

## 8. Weak and strong scaling support

Extra-P can model measurement data from weak and strong scaling experiments. The examples shown in the previous cells all used weak scaling. To create a model using data from a strong scaling experiment, Extra-P uses a workaround. Essentially it converts the data from the strong scaling experiment to a weak scaling experiment. As a consequence metric values, e.g., the runtime increases the larger the jobsize. This might me confusing at first as one would expect the runtime to decrease the larger the jobsize. When analyzing the scalability of the created models one has to think of it as a weak scaling experiment instead of strong scaling.

In [ ]:
data_strong = "../data/lulesh_strong"
thicket_strong = th.Thicket.from_caliperreader(data_strong)
extrap_interface_strong = ExtrapInterface()

### Creating models for strong scaling measurements:

To use this functionality we simply have to set the variable `calc_total_metrics=True` of the `create_models()` function. In addition we need to specify the scaling parameter of the performance experiment, which is usually the resource allocation, e.g., the number of MPI ranks. In this example the *jobsize* corresponds to the number of MPI ranks the application was executed with. Therefore, we set `scaling_parameter="jobsize"` accordingly. Based on this information Extra-P multiplies the measured metric values with the number of MPI ranks, to convert the data from a weak into a strong scaling experiment. Though, this is only done for metrics that are measured per rank, e.g., the *Avg time/rank*. This conversion does not apply to metrics such as the *Total time* for which it would make no sense. If the `scaling` and `scaling_parameter` parameters of the `create_models()` function are not specified, the data will be automatically read as a weak scaling experiment.

In [ ]:
extrap_interface_strong.create_models(thicket_strong, 
                                parameters=[
                                   "jobsize",
                                ], 
                               metrics=[
                                   "Avg time/rank",
                                   "Total time"
                                   ],
                               use_median=True,
                               calc_total_metrics=True,
                               scaling_parameter="jobsize",
                               model_name="config1")

### Analyzing a strong scaling experiment

Subesequently, we can analyze the data as before. We can take a look at thicket's aggregated statistics table or show the models dataframe with the embedded plots.

In [ ]:
thicket_strong.statsframe.dataframe

When analyzing the models dataframe with the embedded plots, we see that models for the metrics *Avg time/rank, Total time*. are almost perfectly identical. This should be the case and indicates that the strong scaling data was correctly converted into a weak scaling experiment.

In this case both metrics are showing redundant information as we intentionally measured the total time to highlight the conversion process. In reality the *Total time* might not be available as a metric. 

Subsequently, one can analyze and display the created models as before.

In [ ]:
model_obj = thicket_strong.statsframe.dataframe.at[thicket_strong.statsframe.dataframe.index[0], "Avg time/rank_extrap-model"]
plt.clf()
fig, ax = model_obj.display(show_mean=True, show_median=True, 
                            show_min_max=True, RSS=True, 
                            AR2=True, show_opt_scaling=True,
                            opt_scaling_func="p**1*log2(p)**1")
ax.legend(loc=1)
plt.show()
plt.close()

## 9. Extra-P Modeler Configuration Support

Extra-P feastures several modeling techniques that are used to create performance models. Depending on the modeling problem at hand one might perform better than another. More information about this can be found in the Extra-P documentation at: [Extra-P](https://github.com/extra-p/extrap).

To show the available modelers from Extra-P that can be used for modeling one can run the below code.

In [ ]:
data = "../data/lulesh/"
thicket = th.Thicket.from_caliperreader(data)
extrap_interface = ExtrapInterface()
extrap_interface.print_modelers()

Furthermore, each modeler has a specific set of configuration options that determine how the models are created and for example define the search space for the models.
To query these options for a specific modeler one can use the following code.

In [ ]:
extrap_interface.print_modeler_options("default")

If you want to explore Extra-P's modeling capabilities you can try to use a different modeler and see the difference in models it will return. To change the model generator Extra-P will use specify the name of the modeler as shown below. Here we use the refining modeler instead, which automatically adjusts the search space of the models for the user. Therefore, there is no need to specify the exponents Extra-P should use for modeling.
Furthermore, this example shows how to create models for several parameters and metrics.
To create models for several parameters or metrics simply provide them as a list of string values.

There are several additional parameters to this method. For example, one can use the parameter `use_median=True` to switch between using the median and mean values of the measured performance metrics values of a measurement point (application configuration) for modeling.

The `add_stats=True` parameter let's you specify if you want to have Extra-P's statistical values extended to the dataframe object of thicket. These values are internally used by Extra-P to decide which of the found hypotheses will be the best model for a certain node/kernel and metric.

In [ ]:
extrap_interface.create_models(thicket, 
                               parameters=[
                                   "jobsize"
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)",
                                   "Avg time/rank"
                                   ], 
                               use_median=True,
                               modeler="refining",
                               model_name="config1")

For advanced users, you can specific which options Extra-P should use when creating the models. This will influence the type of models you will get.
Define the modeler options that you want to use for modeling in the form of a dictonary and pass them to the create_models() function of the extrap interface.
Use the previously shown `print_modeler_options("default")` function to display which options are available for a specific modeler.
Make sure that the options you are setting are available for the modeler that you specified via `modeler=""`. If not the Extra-P interface will let you know and continue using the default options.

In [ ]:
modeler_options = {'allow_log_terms': True,
                   'use_crossvalidation': True,
                   'compare_with_RSS': False,
                   'poly_exponents': "0,1,2,3,4,5",
                   'log_exponents': "0,1,2",
                  }

extrap_interface.create_models(thicket, 
                               parameters=[
                                   "jobsize"
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)",
                                   ], 
                               use_median=True,
                               modeler="default",
                               model_name="config2",
                               modeler_options=modeler_options)

Then one can display the models as before using the following code. Now the datframe has a multi-column index, using the name of the specified `model_name` parameter of the `create_models` function.
Take a look how one of the configurations has performance models for two metrics and the other one has only one. It is also possible to consider differen model parameters for each modeling configuration.

In [ ]:
thicket.statsframe.dataframe

Using the multi-column index one can also simply only analyze one of the configurations used for modeling with Extra-P as shown below.

In [ ]:
thicket.statsframe.dataframe["config2"]

The same goes for displaying the models. By specifying the name of the configuration we can access only the model configuration we want to analyse using the multi-column index.

In [ ]:
model_obj = thicket.statsframe.dataframe["config2"].at[thicket.statsframe.dataframe.index[len(thicket.statsframe.dataframe.index)-1], "Avg time/rank (exc)_extrap-model"]
plt.clf()
fig, ax = model_obj.display(show_mean=True, show_median=True, 
                            show_min_max=True, RSS=True, 
                            AR2=True, show_opt_scaling=True,
                            opt_scaling_func="log2(p)**1")
plt.show()
plt.close()


### Modeler Configuration for Multi-Parameter Models

For multi-parameter models more modeler configuration options are available. First, one can query the options of our multi-parameter modeler as shown below.

In [ ]:
extrap_interface.print_modeler_options("multi-parameter")

For a multi-parameter modeling scenario one first sets the configuration parameters of the multi-parameter modeler, then the one of the single-parameter modeler used by the multi-parameter modeler.

The options are specified in a dictionary as key value pairs. The code below shows an example. To set the single-parameter modeler used by the multi-parameter modeler one has to set `'#single_parameter_modeler': "default"` in this dictionary. To set the options for the single-parameter modeler one has to set `'#single_parameter_options'` and provide a dictionary for the single-parameter modeler options as the value, e.g., `{'poly_exponents': "0,1,2,3,4,5", 'log_exponents': "0,1,2"}`.

In [ ]:
modeler_options = {'compare_with_RSS': False,
                   'allow_combinations_of_sums_and_products': True,
                   '#single_parameter_modeler': "default",
                   '#single_parameter_options': {'poly_exponents': "0,1,2,3,4,5",
                   'log_exponents': "0,1,2"},
                  }

extrap_interface.create_models(thicket, 
                               parameters=[
                                   "jobsize",
                                   "problem_size"
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)"
                                   ], 
                               use_median=True,
                               modeler="multi-parameter",
                               model_name="config3",
                               modeler_options=modeler_options)

## 10. Aggregation of Kernels in a Thicket Dataframe

Sometimes it might be of interest to aggregate a certain set of kernels together and create one performance function for all of them using Extra-P.

To do so the `ExtrapInterface` class provides a method `produce_aggregated_model()` that takes a thicket as an input. The `produce_aggregated_model()` function will autmatically aggregate all kernels in this thicket by their measured metric values aved in the ModelWrapper objects of each kernel. Then a new Extra-P experiment is created that contains only one kernel, the new aggregated one. Subsequently, a new pandas DataFrame is returned that contains the aggregated kernel Model and so on. The code below provides an example for a kernel aggregation.

In [ ]:
agg_df = extrap_interface_multi.produce_aggregated_model(thicket_multi)
agg_df

We can then visualize the aggregated performance model and compare it to the expected behavior or manipulate the plot as wanted.

In [ ]:
model_obj = agg_df.at[agg_df.index[0], "Avg time/rank (exc)_extrap-model"]
plt.clf()
fig, ax = model_obj.display(show_mean=True, show_median=True, 
                            show_min_max=True, RSS=True, 
                            AR2=True)

plt.show()
plt.close()

## 11. Complexity Analysis with Extra-P

Using the functionalities of Extra-P and Thicket one can easily perform a complexity analysis of the scaling behavior of all nodes in a Thicket.
Therefore, one can use the `complexity_statsframe()` function, which requires only the thicket and the evaluation target as an input. With `eval_targets=[[512]]` one specifies the target scale of the values of the considered model parameters that will be used for the evaluation of the models scaling complexity. The `complexity_statsframe()` function will add three columns to the dataframe: model complexity, the model coefficient, and the growth rank. The column names of these three columns are indexed by the specified evaluation target. The model complexity is the term of the model contributes the most to the asymptotic scaling behavior of the model for the specified evaluation target. The model coefficient is the coefficient of this model term. The growth rank is a ranking of all nodes scaling behavior compared with each other.

Furthermore, one can specify several evaluation targets such as (the three columns described above will be added for each evaluation target):

```
eval_targets=[[512]]
eval_targets=[[512],[1024],...]
```

For multiple parameters:

```
eval_targets=[[512,60]]
eval_targets=[[512,60],[1024,70],...]
```

Subsequently, we can for example sort the nodes by their growth ranks and display the dataframe. This provides some insight in which node has the fastest growing metric value at the target scale.

In [ ]:
extrap_interface_multi.complexity_statsframe(thicket_multi, eval_targets=[[512,60]])
complexity_df = thicket_multi.statsframe.dataframe
complexity_df = complexity_df.sort_values(by=["Avg time/rank (exc)_extrap-model_growth_rank_(512,60)"])
complexity_df

We can pipe this information into hatchet and visualize it as a tree using the following code. In the example below each hatchet node shown the metric value and the model complexity.

In [ ]:
th.median(thicket_multi, columns=["Avg time/rank (exc)"])

print(str(thicket_multi.statsframe.tree(
    metric_column='Avg time/rank (exc)_median', 
    annotation_column="Avg time/rank (exc)_extrap-model_complexity_(512,60)", 
    colormap="RdYlGn", 
    )))


We can also replace the metric value with the calculated growth rank of each node as shown in the example below. Note that the color coding of the tree corresponds to the found model complexity, metric values, and growth ranks. The legend indicates the color coding and their values.

In [ ]:
th.median(thicket_multi, columns=["Avg time/rank (exc)"])

print(str(thicket_multi.statsframe.tree(
    metric_column="Avg time/rank (exc)_extrap-model_growth_rank_(512,60)", 
    annotation_column="Avg time/rank (exc)_extrap-model_complexity_(512,60)", 
    colormap="RdYlGn", 
    invert_colormap=True,
    )))

### Multi Configuration Analysis

The same also works when the dataframe contains multiple Extra-P modeler configurations. However, make sure that when, e.g. sorting the dataframe by growth ranks, you also index the modeler configuration.

In [ ]:
thicket.statsframe.dataframe

In [ ]:

extrap_interface.complexity_statsframe(thicket, eval_targets=[[512]])
complexity_df = thicket.statsframe.dataframe
complexity_df["config1"].sort_values(by=["Avg time/rank_extrap-model_growth_rank_(512)"])
complexity_df

Similarly one has to provide the modeler config name `modeler_config="config1"` when creating a hatchet tree, so that the tree function know where to take the data from.

In [ ]:
th.median(thicket, columns=["Avg time/rank"])

print(thicket.statsframe.tree(
    metric_column='Avg time/rank_median', 
    annotation_column="Avg time/rank_extrap-model_complexity_(512)", 
    colormap="RdYlGn",
    modeler_config="config1",
    ))

## 12. Componentizing a Statsframe for Complexity analysis

Another way to analyze the complexity of the kernels in a thicket is by componentizing the models created by Extra-P and analyzing the complexities found in all kernels.
Therefore, the `ExtrapInterface` features a function `componentize_statsframe()` that takes a thicket as an input and then componentizes the extra-p model for each kernel into its different parts, meaning into its terms, e.g., `c, log2(p), p*log2(p), ...`. The code below provides an example for this type of analysis.

In [ ]:
extrap_interface.componentize_statsframe(thicket)
xp_comp_df = thicket.statsframe.dataframe
xp_comp_df